In [2]:
! pip3 install pyconll

  Using cached https://files.pythonhosted.org/packages/1a/70/1935c770cb3be6e3a8b78ced23d7e0f3b187f5cbfab4749523ed65d7c9b1/requests-2.23.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl
    100% |████████████████████████████████| 133kB 1.1MB/s ta 0:00:01
    100% |████████████████████████████████| 163kB 476kB/s ta 0:00:01   32% |██████████▍                     | 51kB 4.7MB/s eta 0:00:01    97% |███████████████████████████████▎| 153kB 966kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/89/e3/afebe61c546d18fb1709a61bee788254b40e736cff7271c7de5de2dc4128/idna-2.9-py2.py3-none-any.whl


In [21]:
import pyconll
import torchtext.data as tt

In [3]:
AFRIKAANS_TRAIN = 'UD_Afrikaans-AfriBooms/af_afribooms-ud-train.conllu'
AFRIKAANS_DEV = 'UD_Afrikaans-AfriBooms/af_afribooms-ud-dev.conllu'
AFRIKAANS_TEST = 'UD_Afrikaans-AfriBooms/af_afribooms-ud-test.conllu'

In [22]:
# cite https://github.com/soutsios/pos-tagger-bert/blob/master/pos_tagger_bert.ipynb
def make_sentences(path):
    data = pyconll.load_from_file(path)
    sentences = []
    tagged_sentences = []
    for each in data:
        tagged_sentence=[]
        sentence = []
        for token in each:
            if token.upos and token.form:
                tagged_sentence.append(token.upos)
                sentence.append(token.form.lower())
        tagged_sentences.append(tagged_sentence)
        sentences.append(sentence)
    return sentences, tagged_sentences

In [23]:
train_afr, tagged_train_afr = make_sentences(AFRIKAANS_TRAIN)
dev_afr, tagged_dev_afr = make_sentences(AFRIKAANS_DEV)
test_afr, tagged_test_afr = make_sentences(AFRIKAANS_TEST)

In [24]:
print(tagged_train_afr[0])

['DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'ADJ', 'NOUN', 'AUX', 'ADP', 'DET', 'NUM', 'NOUN', 'ADP', 'NOUN', 'PART', 'VERB', 'ADP', 'NOUN', 'PRON', 'ADP', 'DET', 'ADJ', 'NOUN', 'VERB', 'AUX', 'PUNCT']


In [25]:
print("Tagged sentences in train set: ", len(tagged_train_afr))
print("Tagged words in train set:", len([item for sublist in tagged_train_afr for item in sublist]))
print(40*'=')
print("Tagged sentences in dev set: ", len(tagged_dev_afr))
print("Tagged words in dev set:", len([item for sublist in tagged_dev_afr for item in sublist]))
print(40*'=')
print("Tagged sentences in test set: ", len(tagged_test_afr))
print("Tagged words in test set:", len([item for sublist in tagged_test_afr for item in sublist]))
print(40*'*')
print("Total sentences in dataset:", len(tagged_train_afr)+len(tagged_dev_afr)+len(tagged_dev_afr))

Tagged sentences in train set:  1315
Tagged words in train set: 33894
Tagged sentences in dev set:  194
Tagged words in dev set: 5317
Tagged sentences in test set:  425
Tagged words in test set: 10065
****************************************
Total sentences in dataset: 1703


In [30]:
# cite https://github.com/tringm/POSTagger_Pytorch/blob/master/src/util/nlp.py
def build_tag_field(sentences_tokens):
    token_field = tt.Field(tokenize=list, init_token="<bos>", eos_token="<eos>")
    fields = [('tokens', token_field)]
    examples = [tt.Example.fromlist([t], fields) for t in sentences_tokens]
    torch_dataset = tt.Dataset(examples, fields)
    token_field.build_vocab(torch_dataset)
    return token_field
    
def build_vocab_field(sentences_words):
    vocab_field = tt.Field(tokenize=list, init_token="<bos>", eos_token="<eos>")
    fields = [('vocab', vocab_field)]
    examples = [tt.Example.fromlist([t], fields) for t in sentences_words]
    torch_dataset = tt.Dataset(examples, fields)
    vocab_field.build_vocab(torch_dataset)
    return vocab_field

In [32]:
train_afr = build_vocab_field(train_afr)
dev_afr = build_vocab_field(dev_afr)
test_afr = build_vocab_field(test_afr)

tagged_train_afr = build_tag_field(tagged_train_afr)
tagged_dev_afr = build_tag_field(tagged_dev_afr)
tagged_test_afr = build_tag_field(tagged_test_afr)

TypeError: 'Field' object is not iterable